## Using Gradio to wrap a an interface around a Titanic Model

Check out the library on [github](https://github.com/gradio-app/gradio-UI) and see the [getting started](https://gradio.app/getting_started.html) page for more demos.

### Installing and importing

In [1]:
!pip install -q gradio 
!pip install pycaret

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Pycaret to train various ML models
from pycaret.classification import *

###Create preprocessing functions and train the model


In [3]:
def encode_ages(df): # Binning ages 
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    categories = pd.cut(df.Age, bins, labels=False)
    df.Age = categories
    return df

def encode_fares(df): # Binning fares
    df.Fare = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    categories = pd.cut(df.Fare, bins, labels=False)
    df.Fare = categories
    return df

def encode_sex(df):
    mapping = {"male": 0, "female": 1}
    return df.replace({'Sex': mapping})

In [4]:
import warnings
warnings.filterwarnings('ignore')


def transform_features(df):
    df = encode_ages(df)
    df = encode_fares(df)
    df = encode_sex(df)
    return df
# Asyraf added
  # train = data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']]
  # train = transform_features(train)


  # X_all = train.drop(['Survived', 'PassengerId'], axis=1)
  # y_all = train['Survived']

  # num_test = 0.20
  # X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

  # # Pass what model to train
  # # if model== 'RF':
  # clf = RandomForestClassifier()
  # clf.fit(X_train, y_train)
  # predictions = clf.predict(X_test)
  # acc = accuracy_score(y_test, predictions)
  # print("accuracy of Random Forest: ", acc)
  # elif model == 'LR':
  #   clf = RandomForestClassifier()
  #   clf.fit(X_train, y_train)
  #   predictions = clf.predict(X_test)
  #   acc = accuracy_score(y_test, predictions)
  #   print("accuracy of Logistic Regression: ", acc)
  # else:
  #   pass

  # return {'Accuracy of Random Forest': int(acc)}


In [13]:
data = pd.read_csv('https://raw.githubusercontent.com/gradio-app/titanic/master/train.csv')

# data = transform_features(data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']])
# y_out = data['Survived']
s = setup(data, target = 'Survived',feature_selection = True, ignore_features = ['Cabin', 'Parch', 'Ticket', 'SibSp', 'Embarked', 'Name', 'Pclass'])

best = compare_models(n_select = 15)
compare_model_results = pull()


# If want to do manual split train testing 
# train = data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']]
# train = transform_features(train)

# X_all = train.drop(['Survived', 'PassengerId'], axis=1)
# y_all = train['Survived']
# # print('yalll is:',y_all)

# num_test = 0.2
# X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8026,0.8443,0.7228,0.7566,0.7360,0.5791,0.5827,0.079
lightgbm,Light Gradient Boosting Machine,0.7963,0.8335,0.7014,0.7645,0.7250,0.5643,0.5715,0.078
lr,Logistic Regression,0.7866,0.8324,0.6931,0.7385,0.7109,0.5426,0.5468,0.236
ada,Ada Boost Classifier,0.7849,0.8188,0.7308,0.7168,0.7209,0.5463,0.5491,0.090
ridge,Ridge Classifier,0.7834,0.0000,0.6764,0.7401,0.7020,0.5332,0.5385,0.012
lda,Linear Discriminant Analysis,0.7834,0.8341,0.6764,0.7401,0.7020,0.5332,0.5385,0.013
qda,Quadratic Discriminant Analysis,0.7817,0.8010,0.7098,0.7199,0.7110,0.5362,0.5396,0.013
nb,Naive Bayes,0.7801,0.8139,0.7098,0.7171,0.7095,0.5332,0.5367,0.013
rf,Random Forest Classifier,0.7801,0.8259,0.6975,0.7291,0.7072,0.5321,0.5377,0.456
et,Extra Trees Classifier,0.7639,0.8086,0.6678,0.7107,0.6826,0.4957,0.5008,0.456


###Write a prediction function

In [22]:
# Modify fn ,add can pick what model to train on and pass accuracy as well - Asyraf
def predict_survival(sex, age, fare, model):

  df = pd.DataFrame.from_dict({'Sex': [sex], 'Age': [age], 'Fare': [fare]})
  df = encode_sex(df)
  df = encode_fares(df)
  df = encode_ages(df)

  model_index = list(compare_model_results['Model']).index(model)
  model = best[model_index]

  # pred = predict_model(model, df,raw_score=True)
  pred = model.predict_proba(df)[0]
  
  return {'Perishes': pred[0].astype('float64'), 'Survives': pred[1].astype('float64')}


###Build a shareable UI in 4 lines of code!

In [ ]:
sex = gr.inputs.Radio(['female', 'male'], label="Sex")
age = gr.inputs.Slider(minimum=0, maximum=120, default=22, label="Age")
fare = gr.inputs.Slider(minimum=0, maximum=1000, default=100, label="Fare (british pounds)")
# Add radio button to train and can choose what classifier to train
model = gr.inputs.Dropdown(list(compare_model_results['Model']), label="Choose Machine Learning Classifier")

gr.Interface(predict_survival, [sex, age, fare, model], "label", live=True).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://11577.gradio.app
Interface loading below...


#### Your model is now live on the gradio.app link shown above. Go ahead and open that in a new tab!

Please contact us [here](mailto:team@gradio.app) if you have any questions, or [open an issue](https://github.com/gradio-app/gradio-UI/issues/new/choose) at our github repo.
